In [ ]:
import pandas as pd

def load_and_clean_data(file_path):
    df = pd.read_csv(file_path).drop_duplicates()
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.strftime('%I:%M %p')
    df = df.sort_values(by="Branch")
    df["Profit Percentage"] = (df["gross income"] / df["cogs"]) * 100
    df["Total Price"] = df["Unit price"] * df["Quantity"]
    
    # Drop columns except 'Product line' which is needed for pivoting
    df = df.drop(columns=["cogs", "gross margin percentage", "Unit price", "Quantity", "Tax 5%", "gross income", "Payment"])
    
    return df

def pivot_product_prices(df):
    if "Product line" not in df.columns:
        raise KeyError("Product line column is missing!")
    
    df_pivot = df.pivot_table(index="Invoice ID", columns="Product line", values="Total Price", aggfunc="sum")
    df_pivot.columns = [f"Price of {col}" for col in df_pivot.columns]
    
    return df_pivot.reset_index()

def main():
    file_path = r"C:\Users\lasya\Projects\supermarket\raw data.csv"
    df = load_and_clean_data(file_path)
    df_pivot = pivot_product_prices(df)
    
    # Drop 'Product line' after pivoting
    df = df.drop(columns=["Product line"])
    
    df = df.merge(df_pivot, on="Invoice ID", how="left").fillna("-")

    columns_order = ["Invoice ID", "Branch", "City", "Customer type", "Gender", "Date", "Time"] + \
                    [col for col in df.columns if col.startswith("Price of")] + ["Profit Percentage", "Rating"]
    df = df[columns_order]
    return df

if __name__ == "__main__":
    df = main()

    df.replace("-", 0, inplace=True)  # Replace "-" with 0
    numeric_cols = [col for col in df.columns if col.startswith("Price of")]
    
    
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)  # Convert to numeric

# Sum sales for each product category per branch
    sales_per_branch = df.groupby("Branch")[numeric_cols].sum()

# Identify the most sold product per branch
    most_sold_product_per_branch = sales_per_branch.idxmax(axis=1)

# Calculate total profit per branch
    df["Total Profit"] = df[numeric_cols].sum(axis=1) * (df["Profit Percentage"] / 100)
    total_profit_per_branch = df.groupby("Branch")["Total Profit"].sum()
    
    
    output_file = "supermarket_analysis_output.csv"

# Combine the data into one DataFrame
    output_df = sales_per_branch.copy()  # Start with the sales data
    output_df["Most Sold Product"] = most_sold_product_per_branch  # Add most sold product column
    output_df["Total Profit"] = total_profit_per_branch  # Add total profit column

# Save to CSV
    output_df.to_csv(output_file)

    print(f"Output saved to {output_file}")

Output saved to supermarket_analysis_output.csv


C:\Users\lasya\AppData\Local\Temp\ipykernel_13236\3323631901.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", 0, inplace=True)  # Replace "-" with 0
